In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
import pickle

In [2]:
#read given data sets 
df = pd.read_csv('Data_Set.csv')
df.head()

,Data,Label
0,Please perform the following steps :<div><br><...,Not Spam
1,"<div class=""post-text"" itemprop=""text"">\r\n<p>...",Not Spam
2,Could you please give us following &nbsp;detai...,Not Spam
3,you can also use<br><br>import os.path<br>os.p...,Not Spam
4,There is no package as&nbsp;<b>lpadmin.&nbsp;<...,Not Spam


In [3]:
#replace all nan value with empty string
df['Data'].replace('', np.nan, inplace=True)
df['Label'].replace('', np.nan, inplace=True)


In [4]:
'''uncomment to see no of null row
a=np.where(pd.isnull(df))
for i,j in zip(*a):
    print(i,j)
'''    

'uncomment to see no of null row\na=np.where(pd.isnull(df))\nfor i,j in zip(*a):\n    print(i,j)\n'

In [5]:
#drop all row which contain empty strings 
df.drop(df.index[752], inplace=True)
df.drop(df.index[340], inplace=True)
df.drop(df.index[216], inplace=True)


In [6]:
#convert row to numpy arrays
X = df['Data'].values
Y = df['Label'].values


In [7]:
#extract all text from each html text and stored in different list
Data_set_X =[]
Data_set_Y =[]
Data_set = []
for i in range(len(X)):
    try:
        soup= BeautifulSoup(X[i],'html5lib')
        Data_set_X.append(soup.text)
        Data_set_Y.append(Y[i])
    except Exception as e:
        print(i,e)

/home/vikrant/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://forums.fossee.in/question/31/while-creating-up-counter-block-in-python-getting-following-erros-at-cmake/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [8]:
#remove all character from text which are not alpha numeric
escapes = ''.join([chr(char) for char in range(1, 32)])
for i in range(len(Data_set_X)):
    Data_set_X[i]=  ' '.join([word for word in Data_set_X[i].split() if (word.isalnum() or word == '.')])

In [9]:

#Data_set_X[:5]

In [10]:
#split data into training and testing sets
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(Data_set_X,Data_set_Y, test_size=0.2, random_state=10)


In [11]:
#counter vectorize words
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words="english")
vect.fit(train_X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:


X_train_df = vect.transform(train_X)
X_test_df = vect.transform(test_X)
type(X_test_df)



scipy.sparse.csr.csr_matrix

In [13]:
#import and train naive bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
model = MultinomialNB(alpha=1.8)
model.fit(X_train_df,train_y)
pred = model.predict(X_test_df)
accuracy_score(test_y, pred)

0.9459459459459459

In [14]:

print(classification_report(test_y, pred , target_names = ["Ham", "Spam"]))


             precision    recall  f1-score   support

        Ham       0.95      0.99      0.97       175
       Spam       0.50      0.10      0.17        10

avg / total       0.93      0.95      0.93       185



In [15]:
#store model in file
with open('spam.pickle','wb') as f:
    pickle.dump(model,f)
with open('vect.pickle','wb') as v:
    pickle.dump(vect,v)

'''
to open spam classifire
pickle_in = open('spam.pickle','rb')
model = pickle.load(pickle_in)
'''

"\nto open spam classifire\npickle_in = open('spam.pickle','rb')\nmodel = pickle.load(pickle_in)\n"

In [16]:

#Uncomment for testing purpose
def predict_spam(string):
    vect = pickle.load(open('vect.pickle','rb'))
    model = pickle.load(open('spam.pickle','rb'))
    soups = BeautifulSoup(string,'html5lib')
    string = soups.text
    templist = string.split()
    string= ' '.join([word for word in templist if (word.isalnum() or word == '.')])
    return model.predict(vect.transform([string]))
    

    



In [17]:
string ='Please look at Scilab help files. Scilab help is available on your Scilab installation and online. \r\n\r\nApart from that, you could also watch Scilab Spoken Tutorials- http://spoken-tutorial.org/tutorial-search/?search_foss=Scilab&amp;search_language=English\r\n '
predict_spam(string)

array(['Not Spam'], dtype='<U8')

In [18]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train_df,train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
pred1 = clf.predict(X_test_df)
print(accuracy_score(test_y, pred1))
print(classification_report(test_y, pred1 , target_names = ["Ham", "Spam"]))


0.9459459459459459
             precision    recall  f1-score   support

        Ham       0.95      1.00      0.97       175
       Spam       0.00      0.00      0.00        10

avg / total       0.89      0.95      0.92       185



/home/vikrant/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train_df,train_y)
pred2 =neigh.predict(X_test_df)
print(accuracy_score(test_y,pred2))
print(classification_report(test_y, pred2 , target_names = ["Ham", "Spam"]))

0.9459459459459459
             precision    recall  f1-score   support

        Ham       0.95      1.00      0.97       175
       Spam       0.00      0.00      0.00        10

avg / total       0.89      0.95      0.92       185



/home/vikrant/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
